# Open-R1-Distill

## 環境構築

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    !apt update && apt install git-lfs -y

In [ ]:
import logging
import os
import subprocess
import sys

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

NVIDIA_SMI = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
logging.info(NVIDIA_SMI)
logging.info(f"Python {sys.version}")

In [ ]:
%pip install torch==2.6.0 transformers==4.52.3

In [ ]:
%pip install vllm==0.8.5.post1

In [ ]:
# 2.8.3はundefined symbolエラーが発生するため2.7.3をインストール
# https://github.com/Dao-AILab/flash-attention/issues/1832
%pip install https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu12torch2.6cxx11abiFALSE-cp312-cp312-linux_x86_64.whl --no-build-isolation

In [ ]:
%pip install \
    accelerate==1.4.0 \
    async-lru \
    bitsandbytes \
    "distilabel[vllm]" \
    deepspeed==0.16.8 \
    hf_transfer \
    langdetect \
    latex2sympy2_extended \
    liger-kernel \
    "trl[vllm]==0.18.0" \
    math-verify==0.5.2 \
    wandb

In [ ]:
if not os.path.exists("open-r1"):
    %git clone https://github.com/huggingface/open-r1.git

if IN_COLAB:
    %cd /content/open-r1
else:
    %cd /workspaces/open-r1-distill/open-r1

%pip install -e ".[dev]" --no-deps

## Data Generation

In [ ]:
if IN_COLAB:
    %cd /content/open-r1
else:
    %cd /workspaces/open-r1-distill/open-r1

!python pipeline.py

## SFT

In [ ]:
if IN_COLAB:
    %cd /content/open-r1
else:
    %cd /workspaces/open-r1-distill/open-r1

!python src/open_r1/sft.py \
    --config recipes/OpenR1-Distill-7B/sft/config_distill.yaml \
    --model_name_or_path Qwen/Qwen3-0.6B-Base \
    --hub_model_id OpenR1-Distill-0.6B \
    --output_dir data/OpenR1-Distill-0.6B \
    --push_to_hub False \
    --report_to none

## GRPO

In [ ]:
if IN_COLAB:
    %cd /content/open-r1
else:
    %cd /workspaces/open-r1-distill/open-r1

!RANK=0 WORLD_SIZE=1 LOCAL_RANK=0 MASTER_ADDR=localhost MASTER_PORT=12345 python src/open_r1/grpo.py \
    --config recipes/DeepSeek-R1-Distill-Qwen-1.5B/grpo/config_demo.yaml \
    --model_name_or_path Qwen/Qwen3-0.6B-Base \
    --hub_model_id OpenR1-Distill-0.6B \
    --output_dir data/OpenR1-Distill-0.6B \
    --push_to_hub False \
    --report_to none \
    --vllm_mode colocate \
    --per_device_train_batch_size 1 \
    --num_generations 4 \
    --gradient_accumulation_steps 8